In [1]:
# Libraries

import twitter
import pyfpgrowth as FPGrowth
from time import sleep
import re
import pandas as pd
import numpy as np
import scipy as scp
from nltk.corpus import stopwords
import matplotlib as plt

In [2]:
# API Twitter
# Owner: MiguelVelazco16 

ConsumerKey = "********************************"
ConsumerSecret = "********************************"
AccessToken = "********************************"
AccessTokenSecret = "********************************"

# Functions

In [3]:
# Functions 

# =======================================================================================================
# =======================================================================================================
# The next function connect to API created in Twitter and download the information

def getLotsOfTweets(searchStr):
    
    api = twitter.Api(consumer_key = ConsumerKey, consumer_secret = ConsumerSecret,
                      access_token_key = AccessToken, access_token_secret = AccessTokenSecret)         
    resultsPages = []
     
    # We need to specify how many tweets we want, so in this case I donloaded 1,500
    
    for i in range(1,15):
        print ("Fetching page %d" % i)
        searchResults = api.GetSearch(raw_query="q=" + str(searchStr) + "%20 &result_type=mixed &count=100")
        resultsPages.append(searchResults)
        print("Wait 6 seconds for the next fetching.")
        sleep(6)
    
    return resultsPages


# The are two ways for searching the information, the fist one is the next
# ------ Raw query 
#"q=NAICM%20 &result_type=mixed &since=2018-09-01 &count=100 &geocode=19.4332255,-99.1359455,100km")        

#The second one is with the function GetSearch
# ------ Sample with the function
#searchResults = api.GetSearch(searchStr, count = 100,
                               #geocode = "19.2494355, -99.6261881, 100km")
                               #until = "2018-08-15",
                               #geocode=[19.24, -99.62, "1km"])                                      
            
# =======================================================================================================
# =======================================================================================================
# In order to mine the tweets and know the patterns words, first we need to removed the urls (https, www,
# http, etc)

def textParse(bigString):
    urlsRemoved = re.sub('(https:[/][/]|www.)([a-z]|[A-Z]|[0-9]|[/.]|[~])*', '', bigString)    
    listOfTokens = re.split(r'\W*', urlsRemoved)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

# =======================================================================================================
# =======================================================================================================
# The stop words are words whitout a meaning like: el, la, los, que, hubiera, etc (spanish). And to 
# In order to avoid them and get the  words with great mean another step is to remove them from tweets.

def remove_stopwords(string):
    string_no_stop_words = []
    for alfa in string:
        if alfa not in stopwords.words("spanish"):
            string_no_stop_words.append(alfa)
            
    return string_no_stop_words

In [4]:
# In this example will use the topic "Industria 4.0 MX", but we can search about any other topic

tweets = getLotsOfTweets('Industria 4.0 MX')

Fetching page 1
Wait 6 seconds for the next fetching.
Fetching page 2
Wait 6 seconds for the next fetching.
Fetching page 3
Wait 6 seconds for the next fetching.
Fetching page 4
Wait 6 seconds for the next fetching.
Fetching page 5
Wait 6 seconds for the next fetching.
Fetching page 6
Wait 6 seconds for the next fetching.
Fetching page 7
Wait 6 seconds for the next fetching.
Fetching page 8
Wait 6 seconds for the next fetching.
Fetching page 9
Wait 6 seconds for the next fetching.
Fetching page 10
Wait 6 seconds for the next fetching.
Fetching page 11
Wait 6 seconds for the next fetching.
Fetching page 12
Wait 6 seconds for the next fetching.
Fetching page 13
Wait 6 seconds for the next fetching.
Fetching page 14
Wait 6 seconds for the next fetching.


In [5]:
# In order to mine the tweets first we need to create a list 
# and at the same time remove all the urls from them

tweets_append = []

for i in range(14):
    for j in range(100):
        tweets_append.append(textParse(tweets[i][j].text))        

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [6]:
# Once we have the list without the urls the next step si to remove the stop words
# Note: if you if you have not used nltk before, first execute the next command "nltk.download()"

tweets_append_no_stop_words = []

for tweet in tweets_append:            
    tweets_append_no_stop_words.append(remove_stopwords(tweet))

In [12]:
# The threshold is the minimum occurrence, in this case we take 100
# FP-Growth - "https://github.com/evandempsey/fp-growth"

threshold = 100
patterns = FPGrowth.find_frequent_patterns(tweets_append_no_stop_words, threshold)

In [13]:
print("List of terms: " + str(len(patterns)))

List of terms: 309


In [14]:
# At the final I'm loading the dictionary a Pandas Dataframe

pd_patterns = pd.DataFrame.from_dict(patterns, orient = 'index')
pd_patterns = pd_patterns.reset_index(level = 0, inplace = False)
pd_patterns = pd_patterns.rename(columns = {'index': 'keys', 0: 'values'})
pd_patterns = pd_patterns.sort_values(by = ['values'], ascending = False)

In [17]:
# At the final we show the frequency of the words, 
# so we need to go deep in order to make a complete 
# analysis of the data, another good example could be 
# get the data from super market and find the frequency pattern in the items bought 

pd_patterns.head(50)

,keys,values
308,"(industria,)",1162
306,"(gobierno,)",644
303,"(industria, plataforma)",616
302,"(plataforma,)",616
307,"(gobierno, industria)",602
305,"(gobierno, industria, plataforma)",602
304,"(gobierno, plataforma)",602
297,"(alista, industria)",434
294,"(alista,)",434
295,"(alista, plataforma)",434
